In [130]:
import sys
sys.path.append("../")  # 添加项目根目录到路径中

In [131]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=1)

In [132]:
seed = 42
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [133]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 6


h_hyper_prmts["convlayers11"] = {"in_channels": 2048, "out_channels": 2048, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers14"] = {"in_channels": 2048, "out_channels": 1024, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers141"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers1"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.05}


# l_hyper_prmts["linerlayer123"] = {"in_channels":1024, "out_channels":327, "use_bn":True, "drop_rate":0.02}
# l_hyper_prmts["linerlayer31"] = {"in_channels":327, "out_channels":6, "use_bn":False, "drop_rate":0.01}
l_hyper_prmts["linerlayer12334"] = {"in_channels":1024, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12"] = {"in_channels":512, "out_channels":327, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer123"] = {"in_channels":327, "out_channels":6, "use_bn":True, "drop_rate":0.02}
# l_hyper_prmts["linerlayer31"] = {"in_channels":64, "out_channels":6, "use_bn":False, "drop_rate":0.01}


hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [134]:
def loss_bs_matrix(outs, hg, device,weight):
    # fmt: off
    r"""
    对于超图的损失函数的矩阵形式.
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``hg``(`Hypergraph`):  超图对象.  
    """
    # fmt: on
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2
    return loss, loss_1, loss_2

#### 定义用于训练的类Trainer

In [135]:
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.  
    """
    # fmt: on
    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  # train mode | 设置为训练模式
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [136]:
import hgp.utils
G = hgp.utils.from_pickle_to_hypergraph("../data/high")
edges, _ = G.e
G.num_e,G.num_v

(7818, 327)

In [137]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=2048, out_features=2048, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=2048, out_features=1024, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (3): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=1024, out_features=1024, bias=True)
      )
    )
  )
  (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.05, inplace=False)
  (4): Linear(in_features=1024, out_features=512, bias=True)
  (5)

In [138]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = 0.13

In [139]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=4e-4, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(4110):
    if hgnn_trainer.weight > 0:
        hgnn_trainer.weight = hgnn_trainer.weight - 0.000002
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 15.7583740234375
                , loss1: -19.622805786132812
                , loss2: 18.309300231933594


in 10 epoch, average loss: -234.938720703125
                , loss1: -2005.6693359375
                , loss2: 25.76619873046875
in 20 epoch, average loss: -600.376611328125
                , loss1: -4658.68046875
                , loss2: 5.096410751342773
in 30 epoch, average loss: -691.9544921875
                , loss1: -5360.71953125
                , loss2: 4.654164505004883
in 40 epoch, average loss: -733.192578125
                , loss1: -5670.375390625
                , loss2: 3.5418960571289064
in 50 epoch, average loss: -762.304345703125
                , loss1: -5896.798046875
                , loss2: 3.7306949615478517
in 60 epoch, average loss: -776.833251953125
                , loss1: -6006.905078125
                , loss2: 3.3855987548828126
in 70 epoch, average loss: -782.727783203125
                , loss1: -6053.225390625
                , loss2: 3.386400604248047
in 80 epoch, average loss: -785.834326171875
                , loss1: -6078.147265625
              

KeyboardInterrupt: 

In [140]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

2125

In [141]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()


tensor([55., 53., 53., 57., 55., 54.], device='cuda:1', grad_fn=<SumBackward1>)


0.012232415902140673